## Note: I dont have enough compute to train the model, i am getting OOM on colab and kaggle as well. please judge on the basis of the code as i am unable to generate results here.

In [20]:
pip install transformers datasets peft bitsandbytes torch trl huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [3]:
from huggingface_hub import login
login()

In [4]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

In [5]:
# 1. Load the dataset
large_dataset = load_dataset("FinGPT/fingpt-fiqa_qa", split="train")

README.md:   0%|          | 0.00/522 [00:00<?, ?B/s]

(…)-00000-of-00001-ab79bf9300210e98.parquet:   0%|          | 0.00/10.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17110 [00:00<?, ? examples/s]

In [6]:
dataset = large_dataset.select(range(int(0.01 * len(large_dataset))))

print(f"Original dataset size: {len(large_dataset)}")
print(f"Reduced dataset size: {len(dataset)}")

Original dataset size: 17110
Reduced dataset size: 171


In [15]:
# 2. Load tokenizer and model
model_name = "google/gemma-2-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)
# 3. Set padding token if not present
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

# 4. Format the dataset
def format_prompt(example):
    return {
        "text": f"""<|startofinstruction|> {example['instruction']}

Question: {example['input']}

Answer: {example['output']} <|endoftext|>"""
    }

# Process dataset
formatted_dataset = dataset.map(format_prompt)

/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1593: UserWarning: Current model requires 6656 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
# 5. Configure QLoRA
lora_config = LoraConfig(
    r=8,  # rank
    lora_alpha=32,  # scaling factor
    target_modules=["q_proj", "v_proj"],  # target attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply QLoRA to the model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 1,597,440 || all params: 2,615,939,328 || trainable%: 0.0611


In [17]:

# 6. Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

# Tokenize dataset
tokenized_dataset = formatted_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=formatted_dataset.column_names
)

Map:   0%|          | 0/171 [00:00<?, ? examples/s]

In [18]:
# 7. Training arguments
training_args = TrainingArguments(
    output_dir="./gemma-2-2b-it-fingpt-qlora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    max_grad_norm=0.3,
    max_steps=500,  # adjust based on your needs
    warmup_steps=50,
    # fp16=True,
    bf16=False,
    report_to="tensorboard",
    push_to_hub=False,
)

# 8. Initialize SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    peft_config=lora_config,
    # dataset_text_field="text",
    # tokenizer=tokenizer,
    # max_seq_length=512,
)

# 9. Train the model
trainer.train()

# 10. Save the model
trainer.save_model("./gemma-2-2b-it-fingpt-qlora-final")
tokenizer.save_pretrained("./gemma-2-2b-it-fingpt-qlora-final")

# 11. Optional: Push to Hugging Face Hub
# trainer.push_to_hub("your-username/gemma-2-2b-it-fingpt-qlora")

Truncating train dataset:   0%|          | 0/171 [00:00<?, ? examples/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacity of 15.89 GiB of which 3.12 MiB is free. Process 6083 has 15.88 GiB memory in use. Of the allocated memory 15.10 GiB is allocated by PyTorch, and 510.16 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [19]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 2304, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2304, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2304, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_p